# Description

## Méthodologie d’analyse

Les étapes suivantes ont été réalisées pour préparer et analyser les données socio-économiques des communes :

- **Chargement et nettoyage des données** : Les données brutes issues de l’INSEE ont été importées, puis filtrées pour exclure les communes ou colonnes ne respectant pas les critères de confidentialité ou contenant trop de valeurs manquantes.

- **Enrichissement des données** : Un mapping a été effectué pour associer chaque code commune à son libellé, puis une clé unique CITY_ID a été créée pour chaque commune.

- **Sélection des variables pertinentes** : Une analyse en composantes principales (PCA) a permis d’identifier les variables ayant le plus d’incidence sur la variabilité des données. Ces variables ont été sélectionnées pour la suite de l’analyse.

- **Préparation des features** : Un sous-ensemble de variables d’intérêt a été sélectionné manuellement pour constituer la table de features utilisée dans les analyses de similarité et de clustering.

- **Normalisation et réduction de dimension** : Les données ont été normalisées (StandardScaler), puis une réduction de dimension a été appliquée (PCA) pour faciliter les analyses de proximité et de clustering.

- **Recherche de villes similaires** : Un algorithme KNN a été utilisé sur les données réduites pour identifier les communes les plus similaires à Levallois-Perret.

- **Transformation des données pour l’analyse temporelle** :Les données ont été transformées au format long, puis pivotées pour obtenir un tableau où chaque ligne correspond à une commune et une catégorie, et chaque colonne à une année (2010, 2015, 2021). Une colonne supplémentaire a été calculée pour mesurer l’évolution entre 2015 et 2021.

- **Préparation pour l’export et la visualisation** : Les données finales ont été exportées pour une utilisation dans Power BI ou d’autres outils de visualisation.

- **Sources** : 
    - https://www.insee.fr/fr/statistiques/5359146#documentation
    - https://www.insee.fr/fr/metadonnees/source/operation/s2146/documentation-methodologique


# Package

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.neighbors import NearestNeighbors

# Fonctions utilitaires

In [ ]:
# Columns to drop for confidentiality (kept from original)
CONFIDENTIAL_COLS = ['PIMP21','TP6021','TP60AGE121','TP60AGE221','TP60AGE321','TP60AGE421',
                     'TP60AGE521','TP60AGE621','TP60TOL121','TP60TOL221','PACT21','PTSA21',
                     'PCHO21','PBEN21','PPEN21','PPAT21','PPSOC21' ,'PPFAM21' ,'PPMINI21',
                     'PPLOGT21' ,'PIMPOT21' ,'D121','D921','RD21','ETASSMAT22','ETAUTRES22']

YEAR_PREFIX_MAP = {"P10": 2010, "P15": 2015, "P21": 2021}

def load_raw(path, sep=';'):
    return pd.read_csv(path, sep=sep, dtype=str)

def clean_raw(df):
    df = df[~df['NBMENFISC21'].isin(['s', 'nd'])]
    df = df.drop(columns=[c for c in CONFIDENTIAL_COLS if c in df.columns], errors='ignore')
    return df.reset_index(drop=True)

def load_meta(meta_path):
    meta = pd.read_csv(meta_path, sep=';', dtype=str)
    return meta[meta['COD_VAR']=='CODGEO'][['COD_MOD', 'LIB_MOD']]

def build_city_df(df, meta_city):
    df_city = df.merge(meta_city, left_on='CODGEO', right_on='COD_MOD', how='left')
    df_city['CITY_ID'] = df_city['CODGEO'].astype(str) + '_' + df_city['LIB_MOD'].astype(str)
    df_city = df_city.drop(['CODGEO', 'COD_MOD','LIB_MOD'], axis=1, errors='ignore')
    return df_city.reset_index(drop=True).fillna(0)

def select_features_by_prefix(df_city, prefixes=None):
    # plus simple à maintenir : prendre colonnes commençant par les préfixes utiles
    if prefixes is None:
        prefixes = ['P10_','P15_','P21_','C10_','C15_','C21_','ET','P21_POP','P15_POP','P10_POP']
    cols = ['CITY_ID'] + [c for c in df_city.columns if any(c.startswith(p) for p in prefixes)]
    return df_city[[c for c in cols if c in df_city.columns]]

def compute_pca_loadings(df, n_components=None):
    X = df.drop(columns=['CITY_ID'])
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    pca = PCA(n_components=n_components) if n_components else PCA()
    pca.fit(Xs)
    loadings = pd.DataFrame(pca.components_.T, index=X.columns,
                            columns=[f'PC{i+1}' for i in range(pca.components_.shape[0])])
    return loadings

def knn_similar(df_feature, target_city_id, n_neighbors=25, pca_components=50):
    X = df_feature.drop(columns=['CITY_ID']).astype(float)
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    pca = PCA(n_components=min(pca_components, Xs.shape[1]))
    Xp = pca.fit_transform(Xs)
    knn = NearestNeighbors(n_neighbors=n_neighbors+1, metric='euclidean')
    knn.fit(Xp)
    # find target index
    idx = df_feature.index[df_feature['CITY_ID']==target_city_id].tolist()
    if not idx:
        return []
    distances, indices = knn.kneighbors([Xp[idx[0]]])
    # skip first (itself)
    return df_feature['CITY_ID'].iloc[indices[0]].tolist()

def transform_time(df_select_city):
    # melt + extract year & category, puis pivot pour analyses temporelles
    df_long = df_select_city.melt(id_vars=['CITY_ID'], var_name='column', value_name='value')
    df_long['YEAR'] = df_long['column'].str.extract(r'(^P\d{2})')[0].map(YEAR_PREFIX_MAP)
    df_long['CATEGORY'] = df_long['column'].str.extract(r'_(.*)')[0]
    df_long = df_long.drop(columns=['column']).dropna(subset=['YEAR','CATEGORY'])
    df_long['value'] = df_long['value'].astype(float)
    # pivot pour visualisation / export (CITY_ID x YEAR x CATEGORY)
    pbi_prep = pd.pivot_table(df_long, values='value', index=['CITY_ID','YEAR'],
                              columns='CATEGORY', aggfunc='sum').reset_index()
    # also produce df_transformed where CATEGORY are rows and YEAR columns (for evolution calc)
    df_transformed = pd.pivot_table(df_long, values='value', index=['CITY_ID','CATEGORY'],
                                    columns='YEAR', aggfunc='sum').reset_index()
    return df_long.drop_duplicates(), pbi_prep, df_transformed

def compute_evolution(df_transformed, y0=2015, y1=2021):
    if y0 in df_transformed.columns and y1 in df_transformed.columns:
        df_transformed['EVOL_{}_{}'.format(y0%100, y1%100)] = (df_transformed[y1]-df_transformed[y0]) / df_transformed[y0] * 100
    return df_transformed

def prepare_pbi_export(pbi_df, out_path):
    pbi_df.to_csv(out_path, sep=';', index=False)


# Execution du pipeline

In [ ]:
RAW_PATH = r'G:\Mon Drive\Famille Vincent\Steven\CESEL\data\INSEE\dossier_complet\dossier_complet.csv'
META_PATH = r'G:\Mon Drive\Famille Vincent\Steven\CESEL\data\INSEE\dossier_complet\meta_dossier_complet.csv'
PBI_OUT = r''#r'G:\Mon Drive\Famille Vincent\Steven\CESEL\data\pbi_prep.csv'
TARGET_CITY = '92044_Levallois-Perret'

df = load_raw(RAW_PATH)
df = clean_raw(df)
meta_city = load_meta(META_PATH)
df_city = build_city_df(df, meta_city)

df_feature = select_features_by_prefix(df_city)
# optional: inspect PCA loadings to refine selection
loadings = compute_pca_loadings(df_feature, n_components=10)
print("PCA loadings preview:")
display(loadings.head())

similar_city = knn_similar(df_feature, TARGET_CITY, n_neighbors=25, pca_components=50)
print("Similar cities (incl. target):", similar_city)

df_select_city = df_city[df_city['CITY_ID'].isin(similar_city) | df_city['CITY_ID'].isin(
    ['92009_Bois-Colombes','92026_Courbevoie','92033_Garches','92035_La Garenne-Colombes',
     '92051_Neuilly-sur-Seine','92062_Puteaux','92063_Rueil-Malmaison','92073_Suresnes'])]

df_long, pbi_prep, df_transformed = transform_time(df_select_city)
df_transformed = compute_evolution(df_transformed, 2015, 2021)

# export pour PowerBI
prepare_pbi_export(pbi_prep, PBI_OUT)

# correlation rapide
for col in df_feature.columns[1:]:
    df_feature[col] = df_feature[col].astype(float)
df_pt = df_feature.pivot_table(columns='CITY_ID')
leval_rating = df_pt[TARGET_CITY]
city_like_levallois = df_pt.corrwith(leval_rating)
t = pd.DataFrame(city_like_levallois, columns=['Correlation']).reset_index()
display(t.sort_values(by='Correlation', ascending=False).head(20))